# Create User Defined Functions (UDFs) in CAS

In [184]:
## Packages
import swat
import sys
import os
import pandas as pd
import numpy as np

## My custom package to connect to the CAS Server
try:
    from casauth import CASAuth
    print('Imported personal custom CAS auth package')
except:
    print('casauth package not available')


print(f'Python version:{sys.version.split("|")[0]}')
print(f'swat version:{swat.__version__}')
print(f'pandas version:{pd.__version__}')
print(f'numpy version:{np.__version__}')

Imported personal custom CAS auth package
Python version:3.8.16 (default, Mar  2 2023, 03:18:16) [MSC v.1916 64 bit (AMD64)]
swat version:1.13.1
pandas version:1.5.3
numpy version:1.24.3


## Make a Connection to CAS (REQUIRED: MODIFY CONNECTION INFORMATION)

##### To connect to the CAS server you will need:
1. the host name, 
2. the portnumber, 
3. your user name, and your password.

Visit the documentation [Getting Started with SAS® Viya® for Python](https://go.documentation.sas.com/doc/en/pgmsascdc/default/caspg3/titlepage.htm) for more information about connecting to CAS.

**Be aware that connecting to the CAS server can be implemented in various ways, so you might need to see your system administrator about how to make a connection. Please follow company policy regarding authentication.**

In [185]:
##
## Connect to CAS
##

################################
## General connection syntax  ##
################################
# conn = swat.CAS(host, port, username, password)

############################################
## SAS Viya for Learners 3.5 connection   ##
############################################
# hostValue = os.environ.get('CASHOST')
# portValue = os.environ.get('CASPORT')
# passwordToken=os.environ.get('SAS_VIYA_TOKEN')
# conn = swat.CAS(hostname=hostValue, port=portValue, password=passwordToken)


##############################
## My Personal connection   ##
##############################
try:
    path = os.getenv('CAS_CREDENTIALS')
    pem_file = os.getenv('CAS_CLIENT_SSL_CA_LIST')
    conn = CASAuth(path, ssl_ca_list = pem_file)
except:
    print('No connection')
    pass

CAS Connection created


## Enter your connection information to CAS below

In [186]:
## conn = swat.CAS()

## Create the fake data for the demonstration

In [187]:
## Create a pandas DataFrame
df = pd.DataFrame([
                "HighTemp = 83; LowTemp = 55;", 
                "HighTemp = 86; LowTemp = 59;", 
                "HighTemp = 92; LowTemp = 63;", 
                "HighTemp = 91; LowTemp = 65;", 
                "HighTemp = 80; LowTemp = 51;", 
                  ], 
             columns=["Temp"])

## Load the pandas DataFrame to the CAS server as a distributed CAS table
castbl = conn.upload_frame(df, casout = {'name':'TEMPDATA', 'caslib':'casuser', 'replace':True})

NOTE: Cloud Analytic Services made the uploaded file available as table TEMPDATA in caslib CASUSER(Peter.Styliadis@sas.com).
NOTE: The table TEMPDATA has been created in caslib CASUSER(Peter.Styliadis@sas.com) from binary data uploaded to Cloud Analytic Services.


In [188]:
## Load the pandas DataFrame to the CAS server as a distributed CAS table
castbl = conn.upload_frame(df, casout = {'name':'TEMPDATA', 'caslib':'casuser', 'replace':True})

display(conn.tableInfo(caslib = 'casuser'),    ## Display available in-memory CAS tables
        type(castbl),                          ## Display the castbl object type (CASTable)
        castbl.head())                         ## Display 5 rows of the castbl

NOTE: Cloud Analytic Services made the uploaded file available as table TEMPDATA in caslib CASUSER(Peter.Styliadis@sas.com).
NOTE: The table TEMPDATA has been created in caslib CASUSER(Peter.Styliadis@sas.com) from binary data uploaded to Cloud Analytic Services.


,Name,Label,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,TEMPDATA,,5,1,0,utf-8,2023-09-09T00:41:59+00:00,2023-09-09T00:41:59+00:00,2023-09-09T00:41:59+00:00,UTF8,0,0,0,,,0,Peter.Styliadis@sas.com,,2023-09-09T00:41:59+00:00,2.009839e+09
1,GOV_IT_BUDGET_NARROW,,13555,6,0,utf-8,2023-08-16T14:55:21+00:00,2023-08-16T14:55:21+00:00,2023-08-23T20:46:46+00:00,UTF8,0,0,0,gov_it_budget_narrow.sashdat,CASUSER(Peter.Styliadis@sas.com),0,Peter.Styliadis@sas.com,,2023-08-16T14:52:12+00:00,2.007817e+09
2,CARS,2004 Car Data,428,15,0,utf-8,2023-08-25T18:16:23+00:00,2023-08-25T18:16:23+00:00,2023-09-09T00:12:35+00:00,UTF8,0,0,0,,,0,Peter.Styliadis@sas.com,,,NaN


swat.cas.table.CASTable

,Temp
0,HighTemp = 83; LowTemp = 55;
1,HighTemp = 86; LowTemp = 59;
2,HighTemp = 92; LowTemp = 63;
3,HighTemp = 91; LowTemp = 65;
4,HighTemp = 80; LowTemp = 51;


## Create a User Defined Function (UDF)

### Pandas solution
If I wanted to solve this problem by creating specific UDFs in pandas, I might do something like this.

In [189]:
##
## Create function to get desired number (this is nice with Pandas)
##


# Separate statements in a column and obtain the number
# Example: "HighTemp = 83; LowTemp = 55;"
# Solution: 83 | 55
def get_numeric_value(data, column, position):
    return (data[column]
            .str.split(';')          ## Split all statements by ;
            .str[position -1]        ## Get the statement by position (Use 1,2,3 instead of 0,1,2
            .str.split(' ')          ## Split the single statement by a space
            .str[-1]                 ## Pull the last element (the number)
            .astype('int'))          ## Convert the characer number to a numeric column


## Convert fahrenheit to celsius
def convert_f_to_c(data, col):
    temp_column = data[col]
    celsius_temp = ((temp_column - 32) * (5/9)).astype('int')
    return celsius_temp


## Create the final DataFrame
final_df = (df
            .assign(
                 HighTemp_F = get_numeric_value(df,'Temp',1),
                 LowTemp_F = get_numeric_value(df,'Temp',2),
                 HighTemp_C = lambda _df: convert_f_to_c(_df, 'HighTemp_F'),
                 LowTemp_C = lambda _df: convert_f_to_c(_df, 'LowTemp_F')
            )
)

final_df.head()

,Temp,HighTemp_F,LowTemp_F,HighTemp_C,LowTemp_C
0,HighTemp = 83; LowTemp = 55;,83,55,28,12
1,HighTemp = 86; LowTemp = 59;,86,59,30,15
2,HighTemp = 92; LowTemp = 63;,92,63,33,17
3,HighTemp = 91; LowTemp = 65;,91,65,32,18
4,HighTemp = 80; LowTemp = 51;,80,51,26,10


### CAS solution
Remember the CAS solution will run on the CAS cluster for massively parallel processing (MPP)

#### Load the fcmpact action set

In [190]:
conn.loadActionSet('fcmpact')

NOTE: Added action set 'fcmpact'.


[actionset]

 'fcmpact'

+ Elapsed: 0.0087s, user: 0.00831s, sys: 0.00962s, mem: 1.05mb

View available actions.

In [191]:
conn.fcmpact?

Signature:   conn.fcmpact(*args, **kwargs)
Type:        Fcmpact
String form: <swat.cas.actions.Fcmpact object at 0x000001EB67C79EE0>
File:        c:\users\pestyl\anaconda3\lib\site-packages\swat\cas\actions.py
Docstring:  
FCMP

Actions
-------
fcmpact.addprototypes : Adds the PROTO definitions and stores them in a table
fcmpact.addroutines   : Adds the FCMP routines and stores them in a table
fcmpact.loadfcmplibs  : Loads FCMP tables in the session CMP library
fcmpact.loadfcmptable : Loads a single FCMP table into memory
fcmpact.runprogram    : Defines the routine to be run

View the value of hte cmplib option. This option specifies one or more SAS data sets that contain compiler subroutines to include during compilation.

#### Add the location of the CAS table that will contain the UDFs.

View the option that needs to be set to import your UDFs.

In [192]:
(conn.
 listsessopts()['SessOpts']
 .query('Name == "cmplib"')
)

,Name,UsageType,Type,Value,Default Value,Group,Min,Max,Description
13,cmplib,Session,String,,,CMP,0,0,specifies one or more SAS data sets that conta...


Set the option to point to the CAS table that we are creating that will contain the UDFs.

In [193]:
conn.setSessOpt(cmplib='casuser.my_udfs')

+ Elapsed: 0.00758s, user: 0.00498s, sys: 0.00671s, mem: 1.39mb

In [194]:
(conn.
 listsessopts()['SessOpts']
 .query('Name == "cmplib"')
)

,Name,UsageType,Type,Value,Default Value,Group,Min,Max,Description
13,cmplib,Session,String,casuser.my_udfs,,CMP,0,0,specifies one or more SAS data sets that conta...


#### Create the UDF in CAS

Comparing the SAS functions to the Pandas method.

| SAS function | Pandas method |
| --- | --- |
| SCAN | SPLIT + [*n*] |
| INPUT | ASTYPE |

In [165]:
## Create UDF

myUDF = '''
    function get_temp_value(colname $, position);
        
        /* Get the statement by position */
        get_statement_from_position = scan(colname, position,';');
        
        /* Get the number from the string */
        get_number_as_string = scan(get_statement_from_position, -1, ' ');
        
        /* Get the number from the statement and convert to a numeric column */
        convert_string_to_numeric = input(get_number_as_string, 8.);
        
        /* Return numeric value */
        return(convert_string_to_numeric);
        
    endsub;
'''

## Add UDF to functions CAS table
conn.addroutines(routineCode = myUDF,
                 package = "myPackageName",
                 saveTable = True,
                 appendTable = True,
                 funcTable = {'name':"my_udfs", 'caslib':'casuser'})

NOTE: Cloud Analytic Services saved the file MY_UDFS.sashdat in caslib CASUSER(Peter.Styliadis@sas.com).


+ Elapsed: 0.157s, user: 0.0669s, sys: 0.179s, mem: 24.4mb

In [166]:
t = conn.CASTable('my_udfs', caslib = 'casuser')
t.head(100)

,_Key_,Owner,Sequence,Type,Subtype,Name,Continue,NValue,Encoded,Value
0,MYPACKAGENAME,CMP,0.0,Header,Package,,0.0,NaN,NaN,"<L n=""Header""><S n=""Version""><![CDATA[1.1]]></..."
1,F.MYPACKAGENAME.GET_TEMP_VALUE,CMP,0.0,Prototype,FCmp,myPackageName,1.0,NaN,NaN,"<L n=""Prototype""><S n=""Name""><![CDATA[get_temp..."
2,F.MYPACKAGENAME.GET_TEMP_VALUE,CMP,1.0,Prototype,FCmp,myPackageName,2.0,NaN,NaN,"N><N n=""Flag6"">32</N><N n=""Flag7"">0</N><N n=""F..."
3,F.MYPACKAGENAME.GET_TEMP_VALUE,CMP,2.0,Header,Function,,0.0,NaN,NaN,"<L n=""Header""><S n=""Version""><![CDATA[1.1]]></..."
4,F.MYPACKAGENAME.GET_TEMP_VALUE,CMP,3.0,Statement Source,Executable,FUNCTION,0.0,65.0,NaN,"function get_temp_value ( colname $ , position..."
5,F.MYPACKAGENAME.GET_TEMP_VALUE,CMP,4.0,Statement Source,Executable,ASSIGN,0.0,1.0,NaN,"get_statement_from_position = scan ( colname ,..."
6,F.MYPACKAGENAME.GET_TEMP_VALUE,CMP,5.0,Statement Source,Executable,ASSIGN,0.0,1.0,NaN,get_number_as_string = scan ( get_statement_fr...
7,F.MYPACKAGENAME.GET_TEMP_VALUE,CMP,6.0,Statement Source,Executable,ASSIGN,0.0,1.0,NaN,convert_string_to_numeric = input ( get_number...
8,F.MYPACKAGENAME.GET_TEMP_VALUE,CMP,7.0,Statement Source,Executable,return,0.0,1.0,NaN,return ( convert_string_to_numeric ) ;
9,F.MYPACKAGENAME.GET_TEMP_VALUE,CMP,8.0,Statement Source,Executable,endsub,0.0,14.0,NaN,endsub ;


In [167]:
conn.tableInfo(caslib = 'casuser')

,Name,Label,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,TEMPDATA,,5,1,0,utf-8,2023-09-09T00:23:05+00:00,2023-09-09T00:23:05+00:00,2023-09-09T00:23:07+00:00,UTF8,0,0,0,,,0,Peter.Styliadis@sas.com,,2023-09-09T00:23:05+00:00,2.009838e+09
1,MY_UDFS,,13,10,0,utf-8,2023-09-09T00:23:10+00:00,2023-09-09T00:23:10+00:00,2023-09-09T00:23:10+00:00,UTF8,1,0,0,,,0,Peter.Styliadis@sas.com,,,NaN
2,GOV_IT_BUDGET_NARROW,,13555,6,0,utf-8,2023-08-16T14:55:21+00:00,2023-08-16T14:55:21+00:00,2023-08-23T20:46:46+00:00,UTF8,0,0,0,gov_it_budget_narrow.sashdat,CASUSER(Peter.Styliadis@sas.com),0,Peter.Styliadis@sas.com,,2023-08-16T14:52:12+00:00,2.007817e+09
3,POP2021_PROC_PYTHON,,52,6,0,utf-8,2023-08-23T18:18:09+00:00,2023-08-23T18:18:09+00:00,2023-08-23T18:18:10+00:00,UTF8,0,0,0,,,0,Peter.Styliadis@sas.com,,,NaN
4,CARS,2004 Car Data,428,15,0,utf-8,2023-08-25T18:16:23+00:00,2023-08-25T18:16:23+00:00,2023-09-09T00:12:35+00:00,UTF8,0,0,0,,,0,Peter.Styliadis@sas.com,,,NaN
5,FUNCTIONS,,8,10,0,utf-8,2023-09-09T00:15:12+00:00,2023-09-09T00:15:12+00:00,2023-09-09T00:17:22+00:00,UTF8,0,0,0,FUNCTIONS.sashdat,CASUSER(Peter.Styliadis@sas.com),0,Peter.Styliadis@sas.com,,2023-09-09T00:05:38+00:00,2.009837e+09


In [168]:
conn.fileInfo(caslib = 'casuser')

,Permission,Owner,Group,Name,Size,Encryption,Time,ModTime
0,-rwxr-xr-x,sas,sas,cars.sas7bdat,139264,,2023-02-23T14:21:31+00:00,1.992781e+09
1,-rwxr-xr-x,sas,sas,previoussales.sas7bdat,73728,,2023-04-26T20:22:48+00:00,1.998160e+09
2,-rwxr-xr-x,sas,sas,VTI.sashdat,413080,NONE,2022-10-11T13:40:38+00:00,1.981115e+09
3,-rwxr-xr-x,sas,sas,hmeq.sashdat,630384,NONE,2022-10-13T17:56:59+00:00,1.981303e+09
4,-rwxr-xr-x,sas,sas,tsa_claims_raw.csv,34936205,,2023-01-16T13:13:53+00:00,1.989494e+09
5,-rwxr-xr-x,sas,sas,warranty_demo.csv,53297896,,2023-08-08T20:38:15+00:00,2.007146e+09
6,-rwxr-xr-x,sas,sas,warranty_final.sashdat,116879328,NONE,2023-08-08T20:41:02+00:00,2.007146e+09
7,-rwxr-xr-x,sas,sas,cars.parquet,4096,NONE,2022-11-17T14:19:19+00:00,1.984314e+09
8,-rwxr-xr-x,sas,sas,RAND_RETAILDEMO.csv,240072190,,2023-05-30T13:10:38+00:00,2.001071e+09
9,-rwxr-xr-x,sas,sas,warranty_final.csv,43615117,,2023-05-30T13:52:57+00:00,2.001074e+09


If the CAS table with the UDF definition is not loaded you have to load it. You can do that with the loadfcmptable action.

In [169]:
conn.loadfcmptable(table = 'MY_UDFS.sashdat', caslib = 'casuser', replace = True)

NOTE: Cloud Analytic Services made the file MY_UDFS.sashdat available as table MY_UDFS in caslib CASUSER(Peter.Styliadis@sas.com).


+ Elapsed: 0.0731s, user: 0.0552s, sys: 0.08s, mem: 3.7mb

Confirm the CAS table with the UDFs is loaded.

In [170]:
conn.tableInfo(caslib = 'casuser')

,Name,Label,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,TEMPDATA,,5,1,0,utf-8,2023-09-09T00:23:05+00:00,2023-09-09T00:23:05+00:00,2023-09-09T00:23:07+00:00,UTF8,0,0,0,,,0,Peter.Styliadis@sas.com,,2023-09-09T00:23:05+00:00,2.009838e+09
1,MY_UDFS,,13,10,0,utf-8,2023-09-09T00:23:11+00:00,2023-09-09T00:23:11+00:00,2023-09-09T00:23:11+00:00,UTF8,0,0,0,MY_UDFS.sashdat,CASUSER(Peter.Styliadis@sas.com),0,Peter.Styliadis@sas.com,,2023-09-09T00:23:10+00:00,2.009838e+09
2,GOV_IT_BUDGET_NARROW,,13555,6,0,utf-8,2023-08-16T14:55:21+00:00,2023-08-16T14:55:21+00:00,2023-08-23T20:46:46+00:00,UTF8,0,0,0,gov_it_budget_narrow.sashdat,CASUSER(Peter.Styliadis@sas.com),0,Peter.Styliadis@sas.com,,2023-08-16T14:52:12+00:00,2.007817e+09
3,POP2021_PROC_PYTHON,,52,6,0,utf-8,2023-08-23T18:18:09+00:00,2023-08-23T18:18:09+00:00,2023-08-23T18:18:10+00:00,UTF8,0,0,0,,,0,Peter.Styliadis@sas.com,,,NaN
4,CARS,2004 Car Data,428,15,0,utf-8,2023-08-25T18:16:23+00:00,2023-08-25T18:16:23+00:00,2023-09-09T00:12:35+00:00,UTF8,0,0,0,,,0,Peter.Styliadis@sas.com,,,NaN
5,FUNCTIONS,,8,10,0,utf-8,2023-09-09T00:15:12+00:00,2023-09-09T00:15:12+00:00,2023-09-09T00:17:22+00:00,UTF8,0,0,0,FUNCTIONS.sashdat,CASUSER(Peter.Styliadis@sas.com),0,Peter.Styliadis@sas.com,,2023-09-09T00:05:38+00:00,2.009837e+09


### Use the UDF

In [171]:
castbl

CASTable('TEMPDATA', caslib='CASUSER(Peter.Styliadis@sas.com)')

In [172]:
castbl.head()

,Temp
0,HighTemp = 83; LowTemp = 55;
1,HighTemp = 86; LowTemp = 59;
2,HighTemp = 92; LowTemp = 63;
3,HighTemp = 91; LowTemp = 65;
4,HighTemp = 80; LowTemp = 51;


In [173]:
(castbl
 .eval("HighTemp = get_temp_value(Temp,1)", inplace = False)
 .eval("LowTemp = get_temp_value(Temp,2)", inplace = False)
 .head()
)

,Temp,HighTemp,LowTemp
0,HighTemp = 83; LowTemp = 55;,83.0,55.0
1,HighTemp = 86; LowTemp = 59;,86.0,59.0
2,HighTemp = 92; LowTemp = 63;,92.0,63.0
3,HighTemp = 91; LowTemp = 65;,91.0,65.0
4,HighTemp = 80; LowTemp = 51;,80.0,51.0


### Add another UDF

In [174]:
## Create UDF to convert Fahrenheit to Celsius
## Formula: Celsius (°C) = (Temperature in degrees Fahrenheit (°F) - 32) * 5/9
myUDF = '''
    function f_to_c(f_temp);
        
        /* Convert the Fahrenheit temp to Celsius */
        c_temp = round((f_temp - 32) * (5/9));
        
        /* Return celsius value */
        return(c_temp);
        
    endsub;
'''

## Add UDF to the same table
conn.addroutines(routineCode = myUDF,
                 package = "myPackageName",
                 saveTable = True,
                 appendTable = True,
                 funcTable = {'name':"my_udfs", 
                              'caslib':'casuser', 
                              'replace':True})

NOTE: Cloud Analytic Services saved the file MY_UDFS.sashdat in caslib CASUSER(Peter.Styliadis@sas.com).


+ Elapsed: 0.207s, user: 0.228s, sys: 0.426s, mem: 147mb

In [175]:
conn.tableInfo(caslib = 'casuser')

,Name,Label,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,TEMPDATA,,5,1,0,utf-8,2023-09-09T00:23:05+00:00,2023-09-09T00:23:05+00:00,2023-09-09T00:23:12+00:00,UTF8,0,0,0,,,0,Peter.Styliadis@sas.com,,2023-09-09T00:23:05+00:00,2.009838e+09
1,MY_UDFS,,20,10,0,utf-8,2023-09-09T00:23:12+00:00,2023-09-09T00:23:12+00:00,2023-09-09T00:23:12+00:00,UTF8,1,0,0,,,0,Peter.Styliadis@sas.com,,,NaN
2,GOV_IT_BUDGET_NARROW,,13555,6,0,utf-8,2023-08-16T14:55:21+00:00,2023-08-16T14:55:21+00:00,2023-08-23T20:46:46+00:00,UTF8,0,0,0,gov_it_budget_narrow.sashdat,CASUSER(Peter.Styliadis@sas.com),0,Peter.Styliadis@sas.com,,2023-08-16T14:52:12+00:00,2.007817e+09
3,POP2021_PROC_PYTHON,,52,6,0,utf-8,2023-08-23T18:18:09+00:00,2023-08-23T18:18:09+00:00,2023-08-23T18:18:10+00:00,UTF8,0,0,0,,,0,Peter.Styliadis@sas.com,,,NaN
4,CARS,2004 Car Data,428,15,0,utf-8,2023-08-25T18:16:23+00:00,2023-08-25T18:16:23+00:00,2023-09-09T00:12:35+00:00,UTF8,0,0,0,,,0,Peter.Styliadis@sas.com,,,NaN
5,FUNCTIONS,,8,10,0,utf-8,2023-09-09T00:15:12+00:00,2023-09-09T00:15:12+00:00,2023-09-09T00:17:22+00:00,UTF8,0,0,0,FUNCTIONS.sashdat,CASUSER(Peter.Styliadis@sas.com),0,Peter.Styliadis@sas.com,,2023-09-09T00:05:38+00:00,2.009837e+09


In [176]:
c = conn.CASTable('my_udfs', caslib = 'casuser')
c.head(50)

,_Key_,Owner,Sequence,Type,Subtype,Name,Continue,NValue,Encoded,Value
0,MYPACKAGENAME,CMP,0.0,Header,Package,,0.0,NaN,NaN,"<L n=""Header""><S n=""Version""><![CDATA[1.1]]></..."
1,F.MYPACKAGENAME.F_TO_C,CMP,0.0,Prototype,FCmp,MYPACKAGENAME,0.0,NaN,NaN,"<L n=""Prototype""><S n=""Name""><![CDATA[f_to_c]]..."
2,F.MYPACKAGENAME.F_TO_C,CMP,1.0,Header,Function,,0.0,NaN,NaN,"<L n=""Header""><S n=""Version""><![CDATA[1.1]]></..."
3,F.MYPACKAGENAME.F_TO_C,CMP,2.0,Statement Source,Executable,FUNCTION,0.0,65.0,NaN,function f_to_c ( f_temp ) ;
4,F.MYPACKAGENAME.F_TO_C,CMP,3.0,Statement Source,Executable,ASSIGN,0.0,1.0,NaN,c_temp = round ( ( f_temp - 32 ) * ( 5 / 9 ) ) ;
5,F.MYPACKAGENAME.F_TO_C,CMP,4.0,Statement Source,Executable,return,0.0,1.0,NaN,return ( c_temp ) ;
6,F.MYPACKAGENAME.F_TO_C,CMP,5.0,Statement Source,Executable,endsub,0.0,14.0,NaN,endsub ;
7,F.MYPACKAGENAME.F_TO_C,CMP,6.0,Symbol,,_HOSTNAME_,0.0,NaN,NaN,"<L n=""Symbol""><S n=""Name""><![CDATA[_HOSTNAME_]..."
8,F.MYPACKAGENAME.GET_TEMP_VALUE,CMP,7.0,Prototype,FCmp,MYPACKAGENAME,1.0,NaN,NaN,"<L n=""Prototype""><S n=""Name""><![CDATA[get_temp..."
9,F.MYPACKAGENAME.GET_TEMP_VALUE,CMP,8.0,Prototype,FCmp,MYPACKAGENAME,2.0,NaN,NaN,"N><N n=""Flag6"">32</N><N n=""Flag7"">0</N><N n=""F..."


In [177]:
conn.fileInfo(caslib = 'casuser')

,Permission,Owner,Group,Name,Size,Encryption,Time,ModTime
0,-rwxr-xr-x,sas,sas,cars.sas7bdat,139264,,2023-02-23T14:21:31+00:00,1.992781e+09
1,-rwxr-xr-x,sas,sas,previoussales.sas7bdat,73728,,2023-04-26T20:22:48+00:00,1.998160e+09
2,-rwxr-xr-x,sas,sas,VTI.sashdat,413080,NONE,2022-10-11T13:40:38+00:00,1.981115e+09
3,-rwxr-xr-x,sas,sas,hmeq.sashdat,630384,NONE,2022-10-13T17:56:59+00:00,1.981303e+09
4,-rwxr-xr-x,sas,sas,tsa_claims_raw.csv,34936205,,2023-01-16T13:13:53+00:00,1.989494e+09
5,-rwxr-xr-x,sas,sas,warranty_demo.csv,53297896,,2023-08-08T20:38:15+00:00,2.007146e+09
6,-rwxr-xr-x,sas,sas,warranty_final.sashdat,116879328,NONE,2023-08-08T20:41:02+00:00,2.007146e+09
7,-rwxr-xr-x,sas,sas,cars.parquet,4096,NONE,2022-11-17T14:19:19+00:00,1.984314e+09
8,-rwxr-xr-x,sas,sas,RAND_RETAILDEMO.csv,240072190,,2023-05-30T13:10:38+00:00,2.001071e+09
9,-rwxr-xr-x,sas,sas,warranty_final.csv,43615117,,2023-05-30T13:52:57+00:00,2.001074e+09


In [178]:
(castbl
 .eval("HighTempF = get_temp_value(Temp,1)", inplace = False)
 .eval("LowTempF = get_temp_value(Temp,2)", inplace = False)
 .eval("HighTempCelsius = f_to_c(HighTempF)", inplace = False)
 .eval("LowTempCelsius = f_to_c(LowTempF)", inplace = False)
 .head()
)

,Temp,HighTempF,LowTempF,HighTempCelsius,LowTempCelsius
0,HighTemp = 83; LowTemp = 55;,83.0,55.0,28.0,13.0
1,HighTemp = 86; LowTemp = 59;,86.0,59.0,30.0,15.0
2,HighTemp = 92; LowTemp = 63;,92.0,63.0,33.0,17.0
3,HighTemp = 91; LowTemp = 65;,91.0,65.0,33.0,18.0
4,HighTemp = 80; LowTemp = 51;,80.0,51.0,27.0,11.0


## Create the Final CAS Table

In [179]:
(castbl
 .eval("HighTempF = get_temp_value(Temp,1)", inplace = False)
 .eval("LowTempF = get_temp_value(Temp,2)", inplace = False)
 .eval("HighTempCelsius = f_to_c(HighTempF)", inplace = False)
 .eval("LowTempCelsius = f_to_c(LowTempF)", inplace = False)
 .copyTable(casout = {'name':'FinalTable', 'caslib':'casuser'})
)

[caslib]

 'CASUSER(Peter.Styliadis@sas.com)'

[tableName]

 'FINALTABLE'

[casTable]

 CASTable('FINALTABLE', caslib='CASUSER(Peter.Styliadis@sas.com)')

+ Elapsed: 0.0846s, user: 0.0953s, sys: 0.191s, mem: 609mb

In [180]:
conn.tableInfo(caslib = 'casuser')

,Name,Label,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,TEMPDATA,,5,1,0,utf-8,2023-09-09T00:23:05+00:00,2023-09-09T00:23:05+00:00,2023-09-09T00:23:14+00:00,UTF8,0,0,0,,,0,Peter.Styliadis@sas.com,,2023-09-09T00:23:05+00:00,2.009838e+09
1,MY_UDFS,,20,10,0,utf-8,2023-09-09T00:23:12+00:00,2023-09-09T00:23:12+00:00,2023-09-09T00:23:14+00:00,UTF8,1,0,0,,,0,Peter.Styliadis@sas.com,,,NaN
2,FINALTABLE,,5,5,0,utf-8,2023-09-09T00:23:14+00:00,2023-09-09T00:23:14+00:00,2023-09-09T00:23:14+00:00,UTF8,0,0,0,,,0,Peter.Styliadis@sas.com,,,NaN
3,GOV_IT_BUDGET_NARROW,,13555,6,0,utf-8,2023-08-16T14:55:21+00:00,2023-08-16T14:55:21+00:00,2023-08-23T20:46:46+00:00,UTF8,0,0,0,gov_it_budget_narrow.sashdat,CASUSER(Peter.Styliadis@sas.com),0,Peter.Styliadis@sas.com,,2023-08-16T14:52:12+00:00,2.007817e+09
4,POP2021_PROC_PYTHON,,52,6,0,utf-8,2023-08-23T18:18:09+00:00,2023-08-23T18:18:09+00:00,2023-08-23T18:18:10+00:00,UTF8,0,0,0,,,0,Peter.Styliadis@sas.com,,,NaN
5,CARS,2004 Car Data,428,15,0,utf-8,2023-08-25T18:16:23+00:00,2023-08-25T18:16:23+00:00,2023-09-09T00:12:35+00:00,UTF8,0,0,0,,,0,Peter.Styliadis@sas.com,,,NaN
6,FUNCTIONS,,8,10,0,utf-8,2023-09-09T00:15:12+00:00,2023-09-09T00:15:12+00:00,2023-09-09T00:17:22+00:00,UTF8,0,0,0,FUNCTIONS.sashdat,CASUSER(Peter.Styliadis@sas.com),0,Peter.Styliadis@sas.com,,2023-09-09T00:05:38+00:00,2.009837e+09


In [181]:
finalTbl = conn.CASTable('FINALTABLE', caslib = 'casuser')
finalTbl.head()

,Temp,HighTempF,LowTempF,HighTempCelsius,LowTempCelsius
0,HighTemp = 83; LowTemp = 55;,83.0,55.0,28.0,13.0
1,HighTemp = 86; LowTemp = 59;,86.0,59.0,30.0,15.0
2,HighTemp = 92; LowTemp = 63;,92.0,63.0,33.0,17.0
3,HighTemp = 91; LowTemp = 65;,91.0,65.0,33.0,18.0
4,HighTemp = 80; LowTemp = 51;,80.0,51.0,27.0,11.0


## Use the Function in SAS Code

One the UDFs are creating using the action, you can use them in other languages that work with the CAS server.


### SAS

IN SAS Studio:

cas conn;

libname casuser cas caslib='casuser';

/* Create fake data */
data person;
   infile datalines delimiter=','; 
   input name $ dept $;
   datalines;                      
HighTemp = 83 LowTemp = 55
;
run;

data casuser.tempdata;
Temp = 'HighTemp = 83; LowTemp = 55;';
output;
Temp = 'HighTemp = 86; LowTemp = 59;';
output;
Temp = 'HighTemp = 92; LowTemp = 63;';
output;
Temp = 'HighTemp = 91; LowTemp = 65;';
output;
Temp = 'HighTemp = 80; LowTemp = 51;';
output;
run; 

proc cas;
	fcmpact.loadfcmptable / table = 'MY_UDFS.sashdat', caslib = 'casuser', replace = True;
quit;

options cmplib=('casuser.my_udfs');

data casuser.final_sas;
    set casuser.tempdata;
    HighTempF = get_temp_value(Temp,1);
    LowTempF = get_temp_value(Temp,2);
    HighTempCelsius = f_to_c(HighTempF);
    LowTempCelsius = f_to_c(LowTempF);
run;

In [182]:
myDataStep = '''
data casuser.final_sas;
    set casuser.tempdata;
    HighTempF = get_temp_value(Temp,1);
    LowTempF = get_temp_value(Temp,2);
    HighTempCelsius = f_to_c(HighTempF);
    LowTempCelsius = f_to_c(LowTempF);
run;
'''
conn.runCode(code = myDataStep)

[InputCasTables]

                              casLib      Name  Rows  Columns                                           casTable
 0  CASUSER(Peter.Styliadis@sas.com)  tempdata     5        1  CASTable('tempdata', caslib='CASUSER(Peter.Sty...

[OutputCasTables]

                              casLib       Name  Rows  Columns  Append Promoted                                           casTable
 0  CASUSER(Peter.Styliadis@sas.com)  final_sas     5        5     NaN        N  CASTable('final_sas', caslib='CASUSER(Peter.St...

+ Elapsed: 0.161s, user: 0.375s, sys: 0.514s, mem: 144mb

In [183]:
newcastbl = conn.CASTable('final_sas', caslib = 'casuser')
newcastbl.head()

,Temp,HighTempF,LowTempF,HighTempCelsius,LowTempCelsius
0,HighTemp = 83; LowTemp = 55;,83.0,55.0,28.0,13.0
1,HighTemp = 86; LowTemp = 59;,86.0,59.0,30.0,15.0
2,HighTemp = 92; LowTemp = 63;,92.0,63.0,33.0,17.0
3,HighTemp = 91; LowTemp = 65;,91.0,65.0,33.0,18.0
4,HighTemp = 80; LowTemp = 51;,80.0,51.0,27.0,11.0


## Terminate the CAS session

In [ ]:
conn.terminate()